# Experimentos - Plataforma de Jurisprudência

Este notebook contém experimentos e testes para a plataforma de jurisprudência.

## 1. Setup Inicial

In [ ]:
import sys
from pathlib import Path

# Adiciona o diretório raiz ao path
sys.path.append(str(Path.cwd().parent))

# Imports
from src.scraper.tjsp_scraper import TJSPScraper
from src.processing.pdf_processor import PDFProcessor
from src.processing.text_chunker import TextChunker
from src.rag.embeddings import EmbeddingsManager
from src.rag.search_engine import JurisprudenceSearchEngine

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configurações de visualização
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

## 2. Teste do Scraper

In [ ]:
# Inicializa o scraper
scraper = TJSPScraper()

# Busca simples
query = "contrato de locação inadimplemento"
print(f"Buscando por: {query}")

# Realiza busca (sem download)
results = scraper.search_jurisprudence(query, max_results=5)

# Mostra resultados
for i, result in enumerate(results):
    print(f"\n--- Resultado {i+1} ---")
    print(f"Processo: {result.get('case_number', 'N/A')}")
    print(f"Data: {result.get('judgment_date', 'N/A')}")
    print(f"Tribunal: {result.get('court', 'N/A')}")

## 3. Teste de Processamento de PDF

In [ ]:
# Inicializa o processador
processor = PDFProcessor()

# Lista PDFs disponíveis
pdf_files = list(processor.raw_dir.glob("*.pdf"))
print(f"PDFs encontrados: {len(pdf_files)}")

if pdf_files:
    # Processa o primeiro PDF
    pdf_path = pdf_files[0]
    print(f"\nProcessando: {pdf_path.name}")
    
    result = processor.process_pdf(pdf_path)
    
    if result:
        print("\nMetadados extraídos:")
        for key, value in result['metadata'].items():
            print(f"  {key}: {value}")
        
        print(f"\nTexto extraído (primeiros 500 caracteres):")
        print(result['cleaned_text'][:500] + "...")

## 4. Teste de Chunking

In [ ]:
# Inicializa o chunker
chunker = TextChunker(chunk_size=500, chunk_overlap=50)

# Texto de exemplo
sample_text = """
ACÓRDÃO

Vistos, relatados e discutidos estes autos de Apelação Cível nº 1234567-89.2023.8.26.0100,
da Comarca de São Paulo, em que é apelante JOÃO DA SILVA, é apelado MARIA DOS SANTOS.

ACORDAM, em 5ª Câmara de Direito Privado do Tribunal de Justiça de São Paulo,
proferir a seguinte decisão: "Negaram provimento ao recurso. V.U.", de conformidade
com o voto do Relator, que integra este acórdão.

O julgamento teve a participação dos Exmos. Desembargadores MOREIRA VIEGAS (Presidente),
JAMES SIANO E FERNANDA GOMES CAMACHO.

São Paulo, 15 de março de 2023.

MOREIRA VIEGAS
RELATOR
"""

# Cria chunks
chunks = chunker.chunk_text(sample_text, metadata={'case_number': '1234567-89.2023.8.26.0100'})

print(f"Número de chunks criados: {len(chunks)}")
print("\nChunks:")
for i, chunk in enumerate(chunks):
    print(f"\n--- Chunk {i+1} ---")
    print(f"Tamanho: {len(chunk.page_content)} caracteres")
    print(f"Conteúdo: {chunk.page_content[:100]}...")
    print(f"Metadados: {chunk.metadata}")

## 5. Teste de Embeddings e Busca

In [ ]:
# Inicializa o gerenciador de embeddings
embeddings_manager = EmbeddingsManager()

# Cria documentos de exemplo
from langchain.schema import Document

sample_docs = [
    Document(
        page_content="Trata-se de ação de cobrança referente a contrato de locação comercial inadimplido.",
        metadata={"case_number": "111111", "type": "cobrança"}
    ),
    Document(
        page_content="Ação de despejo por falta de pagamento de aluguéis vencidos.",
        metadata={"case_number": "222222", "type": "despejo"}
    ),
    Document(
        page_content="Contrato de compra e venda de imóvel com cláusula de retrovenda.",
        metadata={"case_number": "333333", "type": "compra_venda"}
    ),
]

# Adiciona documentos
print("Adicionando documentos ao vector store...")
ids = embeddings_manager.add_documents(sample_docs)
print(f"Documentos adicionados com IDs: {ids}")

# Teste de busca
query = "contrato de aluguel não pago"
print(f"\nBuscando por: '{query}'")

results = embeddings_manager.search(query, k=3)

print("\nResultados:")
for i, result in enumerate(results):
    print(f"\n{i+1}. Score: {1 - result['distance']:.3f}")
    print(f"   Conteúdo: {result['content']}")
    print(f"   Metadados: {result['metadata']}")

## 6. Teste do Search Engine com RAG

In [ ]:
# Inicializa o search engine
# Nota: Certifique-se de ter configurado a API key
try:
    search_engine = JurisprudenceSearchEngine(llm_provider="openai")
    
    # Pergunta de teste
    question = "Quais são os requisitos para uma ação de despejo?"
    print(f"Pergunta: {question}")
    
    # Busca resposta
    response = search_engine.answer_question(question, k=3)
    
    print("\nResposta:")
    print(response['answer'])
    
    print("\nFontes consultadas:")
    for i, source in enumerate(response['sources']):
        print(f"\n{i+1}. {source['metadata']}")
        print(f"   {source['content']}")
        
except Exception as e:
    print(f"Erro ao inicializar search engine: {e}")
    print("Verifique se a API key está configurada no arquivo .env")

## 7. Análise de Performance

In [ ]:
import time

# Teste de performance de embedding
test_texts = [
    "Contrato de locação residencial",
    "Ação de cobrança por inadimplemento contratual",
    "Recurso de apelação em ação de despejo",
    "Sentença procedente em ação de danos morais",
    "Acordo judicial homologado pelo juízo"
]

# Mede tempo de embedding
print("Testando performance de embeddings...")
times = []

for text in test_texts:
    start = time.time()
    embedding = embeddings_manager.embed_query(text)
    end = time.time()
    times.append(end - start)
    print(f"'{text[:30]}...' - {(end-start)*1000:.2f}ms")

print(f"\nTempo médio: {sum(times)/len(times)*1000:.2f}ms")
print(f"Dimensão do embedding: {len(embedding)}")

## 8. Visualização de Similaridades

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Textos para comparar
texts = [
    "Contrato de locação comercial inadimplido",
    "Ação de despejo por falta de pagamento",
    "Contrato de compra e venda de imóvel",
    "Recurso de apelação em matéria criminal",
    "Habeas corpus preventivo"
]

# Gera embeddings
embeddings = [embeddings_manager.embed_query(text) for text in texts]
embeddings_array = np.array(embeddings)

# Calcula matriz de similaridade
similarity_matrix = cosine_similarity(embeddings_array)

# Visualiza
plt.figure(figsize=(8, 6))
sns.heatmap(similarity_matrix, 
            xticklabels=[t[:20] + "..." for t in texts],
            yticklabels=[t[:20] + "..." for t in texts],
            annot=True, 
            fmt=".2f",
            cmap="YlOrRd")
plt.title("Matriz de Similaridade entre Documentos")
plt.tight_layout()
plt.show()

## 9. Estatísticas da Base de Dados

In [ ]:
# Obtém estatísticas
stats = embeddings_manager.get_collection_stats()

print("=== Estatísticas da Base de Dados ===")
for key, value in stats.items():
    print(f"{key}: {value}")

# Conta arquivos
from config.settings import RAW_PDF_DIR, PROCESSED_DIR

pdf_count = len(list(RAW_PDF_DIR.glob("*.pdf")))
processed_count = len(list(PROCESSED_DIR.glob("*.json")))

print(f"\nPDFs baixados: {pdf_count}")
print(f"Documentos processados: {processed_count}")

# Gráfico de barras
data = {
    'PDFs': pdf_count,
    'Processados': processed_count,
    'Vetorizados': stats['document_count']
}

plt.figure(figsize=(8, 5))
plt.bar(data.keys(), data.values(), color=['#1f77b4', '#ff7f0e', '#2ca02c'])
plt.title('Status do Pipeline de Processamento')
plt.ylabel('Quantidade')
plt.grid(axis='y', alpha=0.3)
plt.show()

## 10. Exemplo Completo - Pipeline End-to-End

In [ ]:
# Pipeline completo de exemplo
print("=== PIPELINE COMPLETO ===")

# 1. Busca
print("\n1. BUSCA NO TJSP")
query = "danos morais consumidor"
print(f"Buscando: {query}")

# Simula busca (em produção, usar scraper.scrape_and_download)
print("[Simulado] 3 documentos encontrados")

# 2. Processamento
print("\n2. PROCESSAMENTO DE TEXTO")
sample_text = """
EMENTA: PRESTAÇÃO DE SERVIÇOS - INDENIZAÇÃO POR DANOS MORAIS - 
Falha na prestação de serviços - Responsabilidade objetiva do fornecedor - 
Dano moral configurado - Quantum indenizatório fixado em R$ 5.000,00 - 
Sentença mantida - Recurso não provido.
"""

# 3. Chunking
print("\n3. CHUNKING")
chunks = chunker.chunk_text(sample_text, {'case_number': 'EXEMPLO-001'})
print(f"Criados {len(chunks)} chunks")

# 4. Indexação
print("\n4. INDEXAÇÃO VETORIAL")
ids = embeddings_manager.add_documents(chunks)
print(f"Indexados com IDs: {ids}")

# 5. Busca semântica
print("\n5. BUSCA SEMÂNTICA")
search_query = "indenização por falha no serviço"
results = embeddings_manager.search(search_query, k=1)

if results:
    print(f"Melhor resultado (score: {1 - results[0]['distance']:.3f}):")
    print(results[0]['content'])

print("\n=== PIPELINE CONCLUÍDO ===")